# Aprendizaje basado en instancias

## Clasificador *nokNN*

En el artículo [1] se propone una alternativa al procedimiento de clasificación *kNN* que no se vea sesgada por la elección del número de vecinos ($k$). La idea consiste en agregar los vecindarios obtenidos por el procedimiento estándar del algoritmo *kNN*, para todos los valores de $k$ desde 1 hasta un valor máximo preestablecido ($T_k$); y realizar la clasificación a partir de estos vecindarios. La descripción detallada del procedimiento es la siguiente:

Supongamos que $X$ es el conjunto de entrenamiento e $Y$, la clasificación asociada, definimos
* $N = |X|$, el número de elementos de $X$
* $N_c = |\{x \in X, Y(x) = c\}|$, el número de elementos de $X$ con valor de clasificación $c$
* $M_c = (2^N-1) \cdot N_c/N$

Dado el valor $T_k$ preestablecido, se consideran los conjuntos $A_k \subseteq X$, para todo $k = 1,..,T_k$. Cada $A_k$ es el vecindario formado por los $k$ vecinos más cercanos a un nuevo valor $x$ dentro de $X$.

Para cada $A_k$ se define su *función de masa* para la clase $c$ de la siguiente forma:

$$m_c(A_k) = \frac{P(c|A_k)}{M_c} = \frac{P(A_k,c)}{P(A_k)\cdot M_c} = 
             \frac{|\{x \in A_k, Y(x) = c\}|}{k\cdot M_c}$$

Por completitud se supone la existencia de otro conjunto de datos $A \subseteq X$ tal que $m_c(A)$ compense los valores de $m_c(A_k)$ para que la suma total sea 1, tal y como se indica en [1].

A continuación se define una función de probabilidad basada en la *función de masa*

$$G_c(x) = \sum_{B\subseteq X} m_c(B) \cdot \frac{|\{x\}\cap B|}{|B|}$$

Suponiendo además que el elemento $x$ está en cada uno de los vecindarios $A_h$, pero no está en el conjunto $A$ y dado que el valor de la *función de masa* de cualquier conjunto distinto de $A_h$ y $A$ es 0, se simplifica el cálculo de $G_c(x)$ de la siguiente manera:

$$G_c(x) = \sum_{k=1}^{T_k} m_c(A_k) \cdot \frac{|\{x\}\cap A_k|}{|A_k|} = 
           \sum_{k=1}^{T_k} m_c(A_k) \cdot \frac{1}{k} =
           \sum_{k=1}^{T_k} \frac{|\{x \in A_k, Y(x) = c\}|}{k\cdot M_c} \cdot \frac{1}{k} =
           \sum_{k=1}^{T_k} \frac{|\{x \in A_k, Y(x) = c\}|}{k^2\cdot M_c}$$
           
Finalmente, el valor de clasificación de $x$ será la clase $c$ para la que el valor $G_c(x)$ sea máximo.

[1] Wang, H., Düntsch, I., Gediga, G., Guo, G. (2005). Nearest Neighbours without k . In: Monitoring, Security, and Rescue Techniques in Multiagent Systems. Advances in Soft Computing, vol 28. Springer, Berlin, Heidelberg. https://doi.org/10.1007/3-540-32370-8_12

## Contenido del ejercicio

El ejercicio consiste en:

* Definir la función `kNNBestKScore(T_k,X_train,y_train,X_test,y_test)` que, dado un valor máximo para el número de vecinos, `T_k`, un conjunto de entrenamiento y su clasificación, `X_train` e `y_train`, y un conjunto de prueba y su clasificación, `X_test` e `y_test`, devuelva el par `(maxK,maxS)` formado por el valor del número de vecinos, `maxK`, entre 1 y `T_k` (incluido) que maximiza el rendimiento en el conjunto de prueba `X_test` del clasificador *kNN* construido con el conjunto de entrenamiento `X_train`, siendo `maxS` dicho rendimiento.
* Definir la función `nokNNClassifierPredict(T_k,X_train,y_train,X)` que, dado un valor máximo para el número de vecinos, `T_k`, un conjunto de entrenamiento y su clasificación, `X_train` e `y_train`, y un array de valores nuevos, `X`, devuelve el array de clasificaciones asignadas a los valores del array `X` por el clasificador *nokNN* descrito en el primer apartado de este enunciado.
* Definir la función `nokNNClassifierScore(T_k,X_train,y_train,X_test,y_test)` que, dado un valor máximo para el número de vecinos, `T_k`, un conjunto de entrenamiento y su clasificación, `X_train` e `y_train`, y un conjunto de prueba y su clasificación, `X_test` e `y_test`, devuelve el rendimiento en el conjunto de prueba `X_test` del clasificador *nokNN* construido con el conjunto de entrenamiento `X_train`.
* Evaluar las funciones `kNNBestKScore` y `nokNNClassifierScore` sobre los conjuntos de datos *Iris*, *Wine* y *Breast Cancer* de *Scikit-Learn*.

El **desarrollo tiene que estar razonado**, indicando en cada apartado qué se está haciendo, **demostrando así el conocimiento adquirido en este módulo**. ¿Qué conclusiones puedes sacar sobre el clasificador *nokNN* comparado con el *kNN*?

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.metrics import accuracy_score
import numpy as np 

In [2]:
# Aqui simplemente cargamos los conjuntos de datos en 3 variables: iris, wine y cancer

from sklearn.datasets import load_iris,load_wine,load_breast_cancer

iris = load_iris()

wine= load_wine()

cancer = load_breast_cancer()

# Definimos el conjunto Iris

Aunque para el apartado de la comparacion usare los 3 conjuntos,igualmente en cada apartado haré una llamada  y evaluraé el método para comprobar su correcto funcionamiento


In [3]:
X_data, y_data, X_names, y_names = \
    iris.data, iris.target, iris.feature_names, iris.target_names

X_train, X_test, y_train, y_test = \
  train_test_split(X_data,y_data,test_size = 0.33,
                   random_state=4861,stratify=y_data)

## Apartado 1  kNNBestKScore

Definir la función `kNNBestKScore(T_k,X_train,y_train,X_test,y_test)` que, dado un valor máximo para el número de vecinos, `T_k`, un conjunto de entrenamiento y su clasificación, `X_train` e `y_train`, y un conjunto de prueba y su clasificación, `X_test` e `y_test`, devuelva el par `(maxK,maxS)` formado por el valor del número de vecinos, `maxK`, entre 1 y `T_k` (incluido) que maximiza el rendimiento en el conjunto de prueba `X_test` del clasificador *kNN* construido con el conjunto de entrenamiento `X_train`, siendo `maxS` dicho rendimiento.

En este apartado tengo que calcular el valor máximo para el conjunto de vecinos, para eso me he definido dos variables maxK y maxS que serán el rendimiento y el nº de vecinos que maximiza este, después recorro los valores desde 1 hasta T_k siendo este el nº de vecinos máximo, creo después una instancia con KNeighborsClassifier para el número de vecinos de la iteración y entreno la instancia con el conjunto de entrenamiento.

Finalmente calculo el rendimiento con el score y lo comparo al almacenado, si es mayor guardo el score y el valor de la iteración (i) como los nuevos máximos.

In [4]:
# T_k es el numero maximo de vecinos
# Tengo que devolver maxK (vecinos que maximiza el rendimiento) y maxS (el rendimiento más alto)
def kNNBestKScore(T_k,X_train,y_train,X_test,y_test):
    
    #Declaro las variables del resultado inicializadas a 0
    maxK=0;
    maxS=0;
    
    #Recorro los valores desde 1 hasta el valor de T_k, que es el número máximo de vecinos
    for i in range(1, T_k + 1):

        # definimos el clasificador  para cada valor de i
        knn1 = KNeighborsClassifier(n_neighbors=i,p=2)
        
        # entrenamos
        knn1.fit(X_train,y_train)
        
        # calculamos el rendimiento para el conjunto de prueba
        score = knn1.score(X_test,y_test)
        
        # comparamos, si es mejor lo guardamos en la variable
        if(score > maxS):
            maxS= score;
            maxK=i;
    
    #Imprimimos por pantalla para dar mas resultado
    print("El número de vecinos que maximiza el rendimiento es " + str(maxK) + " cuyo rendimiento es " + str(maxS))
    
    return maxK,maxS

In [5]:
kNNBestKScore(5,X_train,y_train,X_test,y_test)

El número de vecinos que maximiza el rendimiento es 5 cuyo rendimiento es 0.96


(5, 0.96)

## Apartado 2 nokNNClassifierPredict

Definir la función `nokNNClassifierPredict(T_k,X_train,y_train,X)` que, dado un valor máximo para el número de vecinos, `T_k`, un conjunto de entrenamiento y su clasificación, `X_train` e `y_train`, y un array de valores nuevos, `X`, devuelve el array de clasificaciones asignadas a los valores del array `X` por el clasificador *nokNN* descrito en el primer apartado de este enunciado.

Teniendo en cuenta el enunciado de la primera parte, en este apartado lo que he hecho ha sido:

* Primero antes de nada inicializo un array con los valores donde almacenaré las predicciones, además de calcular y almacenar el nº de elementos del conjunto en una variable N

* Después recorro los elementos del conjunto X, en el que calcularemos las distancias

* Itero sobre cada clase unica y sobre los vecindarios de 1 hasta T_k (incluido), en el que calculo los vecinos mas cercanos,pondero por la prob de la clase y almaceno la puntuación para esa clase

* Finalmente me quedo la que tenga un valor máximo

Referencias utilizadas en este apartado:

* https://stackoverflow.com/questions/16680334/using-numpy-argsort-and-take-in-2d-arrays
* https://stackoverflow.com/questions/7741878/how-to-apply-numpy-linalg-norm-to-each-row-of-a-matrix

In [6]:
def nokNNClassifierPredict(T_k, X_train, y_train, X):
    # Inicializamos el array vacio de las futuras predicciones
    y_pred = []
    
    # Calculo el número de elementos en el conjunto de entrenamiento
    N = len(X_train)
    
    for x in X:
        # defino una variable para almacenar G_c(x) para cada clase c
        c_scores = {}

        # Calculamos las distancias
        distances = np.linalg.norm(X_train - x, axis=1)

        # itero sobre cada clase unica del conjunto 
        for c in np.unique(y_train):
            # calculamos los elementos del conjunto con el valor c
            N_c = np.sum(y_train == c)
            M_c = (2 * N - 1) * N_c / N
            G_c_x = 0.0

            # iteramos sobre cada vecindario
            for k in range(1, T_k + 1):
                
                # calculamos los mas cercanos
                nearest_neighbors_indices = np.argsort(distances)[:k]
                neighbors_c = np.sum(y_train[nearest_neighbors_indices] == c)
                # Ponderamos por la probabilidad de la clase
                G_c_x += (neighbors_c / k) * M_c

            # almacenamos la puntuacion para la clase c
            c_scores[c] = G_c_x

        # Nos quedamos con la clase con el valor máximo
        y_pred.append(max(c_scores, key=c_scores.get))

    return np.array(y_pred)

Cuando ejecuto este método lo que me devuelve es un array con las clasificaciones del conjunto X_test

In [7]:
nokNNClassifierPredict(5, X_train, y_train, X_test)

array([1, 2, 2, 0, 1, 1, 2, 2, 1, 1, 2, 0, 1, 0, 1, 1, 2, 2, 0, 0, 2, 1,
       1, 0, 0, 1, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 2, 0, 2,
       2, 0, 1, 0, 2, 1])

## Apartado 3 nokNNClassifierScore

Definir la función `nokNNClassifierScore(T_k,X_train,y_train,X_test,y_test)` que, dado un valor máximo para el número de vecinos, `T_k`, un conjunto de entrenamiento y su clasificación, `X_train` e `y_train`, y un conjunto de prueba y su clasificación, `X_test` e `y_test`, devuelve el rendimiento en el conjunto de prueba `X_test` del clasificador *nokNN* construido con el conjunto de entrenamiento `X_train`.

Aquí lo que estoy haciendo es simplemente llamar al metodo que he creado en el apartado 2 nokNNClassifierPredict y el array con la clasificacion de los valores del conjunto  X_test(y_pred) lo comparo con la del array de y_test con la función accuracy_score obteniendo así el rendimiento(score)


In [8]:
def nokNNClassifierScore(T_k, X_train, y_train, X_test, y_test):
    # llamamos al metodo de noKNN y guardamos las predicciones en una variables
    y_pred = nokNNClassifierPredict(T_k, X_train, y_train, X_test)
    #Calculamos el rendimiento con la función accuracy_score 
    score = accuracy_score(y_test,y_pred)
    
    return score

In [9]:
nokNNClassifierScore(5, X_train, y_train, X_test, y_test)

0.94

## Apartado 4

Evaluar las funciones `kNNBestKScore` y `nokNNClassifierScore` sobre los conjuntos de datos *Iris*, *Wine* y *Breast Cancer* de *Scikit-Learn*

En este apartado voy a evaluar los 3 conjuntos uno a uno y viendo los rendimientos para el mismo conjunto del clasificador KNN y el noKNN que he realizado en el apartado 1 y 2 respectivamente

In [10]:
X_data, y_data, X_names, y_names = \
    iris.data, iris.target, iris.feature_names, iris.target_names

X_train, X_test, y_train, y_test = \
  train_test_split(X_data,y_data,test_size = 0.33,
                   random_state=4861,stratify=y_data)

In [11]:
kNNBestKScore(5,X_train,y_train,X_test,y_test)

nokNNClassifierScore(5, X_train, y_train, X_test, y_test)

El número de vecinos que maximiza el rendimiento es 5 cuyo rendimiento es 0.96


0.94

In [12]:
X_data, y_data, X_names, y_names = \
    wine.data, wine.target, wine.feature_names, wine.target_names

X_train, X_test, y_train, y_test = \
  train_test_split(X_data,y_data,test_size = 0.33,
                   random_state=4861,stratify=y_data)

In [13]:
kNNBestKScore(5,X_train,y_train,X_test,y_test)

nokNNClassifierScore(5, X_train, y_train, X_test, y_test)

El número de vecinos que maximiza el rendimiento es 1 cuyo rendimiento es 0.7457627118644068


0.7627118644067796

In [14]:
X_data, y_data, X_names, y_names = \
    cancer.data, cancer.target, cancer.feature_names, cancer.target_names

X_train, X_test, y_train, y_test = \
  train_test_split(X_data,y_data,test_size = 0.33,
                   random_state=4861,stratify=y_data)

In [15]:
kNNBestKScore(5,X_train,y_train,X_test,y_test)

nokNNClassifierScore(5, X_train, y_train, X_test, y_test)

El número de vecinos que maximiza el rendimiento es 2 cuyo rendimiento es 0.9468085106382979


0.9308510638297872

## Conclusiones

* El rendimiento del noKNN es muy parecido en términos de rendimiento al KNN en los conjuntos de datos que hemos testado

* El algoritmo noKNN no se ve tan afectado por la sensibilidad del parámetro K (infrajuste o sobreajuste) debido a que no está tan condicionada por el valor de K, ya que calcula múltiples valores de este y luego combina la información para la clasificación.

* Para conjuntos de datos más estructurados el KNN debe ser mejor mientrás que para estructuras de datos no tan claras será mejor el noKNN ya que se adapta de manera más flexible a la estructura de datos

* A nivel computacional noKNN debe ser más costoso ya que supone múltiples valores de k
